In [11]:
import bs4
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import JSONLoader
from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
llm = ChatOpenAI(model = 'gpt-4o')

In [4]:
loader = JSONLoader(file_path='Gurshan_Data.json', jq_schema='.', text_content=False)
docs=loader.load()

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size =1000, chunk_overlap= 200,add_start_index=True)
splits = text_splitter.split_documents(docs)

In [6]:
vectorstore = InMemoryVectorStore.from_documents(documents=splits, embedding= OpenAIEmbeddings())

In [7]:
retriever = vectorstore.as_retriever(search_type= 'similarity')

In [12]:
def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]
store = {}

In [14]:
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)


contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


In [15]:
history_aware_retriever = create_history_aware_retriever(llm, retriever, contextualize_q_prompt)


In [16]:
system_prompt = (
    "You are a nurse updating a parent on the status of their child in the NICU."
    "Use the following retrieved context to provide relevant information to the parent. "
    "Speak in a conversational tone, and keep the summary concise."
    "Do not diagnose."
    "\n\n{context}"
)

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [18]:
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [19]:
conversational_rag_chain.invoke(
    {"input": "How is my child"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

"Your little one is being closely monitored in the NICU, and I'm here to update you on their status. Their vital signs, like heart rate and oxygen saturation, have been stable overall. The team has been adjusting feeding amounts and making sure they're comfortable. There have been moments where their skin tone has varied, sometimes appearing blueish or jaundiced, which is common in premature babies. We've also been managing their pain levels, which occasionally have been moderate to high, with appropriate interventions.\n\nWe've been keeping an eye on their temperature, and it's been within a normal range. The medical team has been proactive with interventions like lighting adjustments, position changes, and administering medication when necessary. Please let me know if you have any specific questions or if there's anything else you'd like to know."

In [20]:
conversational_rag_chain.invoke(
    {"input": "Hows he doing with the interventions?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

"Your little one is doing okay with the interventions we're providing in the NICU. We've had to make some adjustments like increasing and then decreasing the oxygen flow to help with breathing. There have been some moments of discomfort, but we are managing those with pain relief measures. The color changes in his skin tone, like the bluish tint, are something we're keeping a close eye on and addressing as needed. We've also given antibiotics and other medications to support his health. Overall, we're carefully monitoring his vital signs and making necessary adjustments to ensure he's as comfortable and stable as possible. If you have any specific concerns or questions, feel free to ask!"

In [21]:
conversational_rag_chain.invoke(
    {"input": "what did I just ask?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]


'You asked about how your premature child is doing in the NICU.'

In [24]:
conversational_rag_chain.invoke(
    {"input": "what is my babies name"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

"I'm sorry, but I can't access personal information like your baby's name. If you have any questions about your baby's condition or care, feel free to ask!"

In [28]:
conversational_rag_chain.invoke(
    {"input": "From what day until what day"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'The data shows a time range from October 8, 2024, to October 9, 2024.'

In [ ]:
def parentChatbotSummarize(filepath: str, promptQuestion: str ):
    llm = ChatOpenAI(model = 'gpt-4o')

    loader = JSONLoader(file_path=filepath, jq_schema='.', text_content=False)
    docs=loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size =1000, chunk_overlap= 200,add_start_index=True)
    splits = text_splitter.split_documents(docs)
    vectorstore = InMemoryVectorStore.from_documents(documents=splits, embedding= OpenAIEmbeddings())
    retriever = vectorstore.as_retriever(search_type= 'similarity')

    system_prompt = (
    "You are a nurse explaining to a parent how their premature child is doing in the NICU. You cannot make a diagnoses, just explain briefly."
    "Use the following retrieved context to provide relevant information to the parent. "
    "Speak in a conversational tone, and keep the summary concise."
    "\n\n{context}"
    )

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("human", "{input}"),
        ]
    )
    question_answer_chain = create_stuff_documents_chain(llm, prompt)
    rag_chain = create_retrieval_chain(retriever, question_answer_chain)
    
    response = rag_chain.invoke({"input": promptQuestion})

    return response['answer']





    